No Place for Hate

We seek to find the correlation of the occurrence of Hate Crimes and:
* Education
* Poverty
* Unemployment Rates




In [2]:
#Import Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress
import plotly.express as px


In [3]:
# import and merge data sets - Education, and Unemployment Rates by state
#read Education csv file and review the sample rows
edu_data = pd.read_csv("Resources/EducationReport.csv")
edu_data.head(3)

,Unnamed: 0,Total,Total.1,Total.2,Total.3,Total.4,Urban,Urban.1,Urban.2,Urban.3,Urban.4,Rural,Rural.1,Rural.2,Rural.3,Rural.4
0,Name,1970,1980,1990,2000,2014-2018,1970,1980,1990,2000,2014-2018,1970,1980,1990,2000,2014-2018
1,United States,10.70%,16.20%,20.30%,24.40%,31.50%,11.50%,17.50%,22.00%,26.20%,33.50%,6.70%,10.50%,12.30%,14.90%,19.60%
2,Alabama,7.80%,12.20%,15.70%,19.00%,24.90%,8.90%,13.70%,17.80%,21.70%,28.00%,5.30%,8.20%,9.80%,11.60%,15.20%


In [4]:
#create a new dataframe with only needed columns for years 2014-2018
edu_data_df = pd.DataFrame(edu_data)
edu_data_mycolm = edu_data_df[["Unnamed: 0", "Total.4", "Urban.4", "Rural.4"]]
edu_data_upd = edu_data_mycolm.rename(columns={"Unnamed: 0" : "area_name", "Total.4" : "Total Edu", "Urban.4" : "Total Edu Urban", "Rural.4" : "Total Edu Rural"})
edu_data_upd.head(5)

,area_name,Total Edu,Total Edu Urban,Total Edu Rural
0,Name,2014-2018,2014-2018,2014-2018
1,United States,31.50%,33.50%,19.60%
2,Alabama,24.90%,28.00%,15.20%
3,Alaska,29.20%,31.80%,23.90%
4,Arizona,28.90%,29.60%,15.80%


In [5]:
unempl_data = pd.read_csv("Resources/Unemployment.csv", encoding="ISO-8859-1")
unempl_data.head(5)

,FIPStxt,Stabr,area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,...,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Median_Household_Income_2018,Med_HH_Income_Percent_of_State_Total_2018
0,0,US,United States,NaN,NaN,NaN,"142,601,667","136,904,680","5,696,987",4.0,...,"161,389,026","155,102,319","6,286,707",3.9,"163,100,055","157,115,247","5,984,808",3.7,"61,937",NaN
1,1000,AL,Alabama,NaN,NaN,NaN,"2,133,223","2,035,594","97,629",4.6,...,"2,216,627","2,130,845","85,782",3.9,"2,241,747","2,174,483","67,264",3.0,"49,881",100.0
2,1001,AL,"Autauga County, AL",2.0,2.0,1.0,"21,720","20,846",874,4.0,...,"26,196","25,261",935,3.6,"26,172","25,458",714,2.7,"59,338",119.0
3,1003,AL,"Baldwin County, AL",3.0,2.0,1.0,"69,533","66,971","2,562",3.7,...,"95,233","91,809","3,424",3.6,"97,328","94,675","2,653",2.7,"57,588",115.5
4,1005,AL,"Barbour County, AL",6.0,6.0,0.0,"11,373","10,748",625,5.5,...,"8,414","7,987",427,5.1,"8,537","8,213",324,3.8,"34,382",68.9


In [6]:
#filter data to get to the state level (the columns where rural urban code & metro 2013 columns are empty)
unempl_data['Rural_urban_continuum_code_2013'].replace('', np.nan, inplace=True)
unempl_state_interim = unempl_data[unempl_data['Rural_urban_continuum_code_2013'].isnull()]
unempl_state_interim
unempl_data_clean = unempl_state_interim[unempl_state_interim['Metro_2013']!=0]
unempl_data_clean.head(3)

,FIPStxt,Stabr,area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,...,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Median_Household_Income_2018,Med_HH_Income_Percent_of_State_Total_2018
0,0,US,United States,NaN,NaN,NaN,"142,601,667","136,904,680","5,696,987",4.0,...,"161,389,026","155,102,319","6,286,707",3.9,"163,100,055","157,115,247","5,984,808",3.7,"61,937",NaN
1,1000,AL,Alabama,NaN,NaN,NaN,"2,133,223","2,035,594","97,629",4.6,...,"2,216,627","2,130,845","85,782",3.9,"2,241,747","2,174,483","67,264",3.0,"49,881",100.0
69,2000,AK,Alaska,NaN,NaN,NaN,"319,511","299,146","20,365",6.4,...,"352,662","329,901","22,761",6.5,"347,779","326,430","21,349",6.1,"74,912",100.0


In [7]:
#select columns needed for years 2014-2018
unempl_data_mycolm = unempl_data_clean[["Stabr", "area_name", "Civilian_labor_force_2018", "Employed_2018", "Unemployed_2018", "Unemployment_rate_2018"]]
unempl_data_mycolm.head()

,Stabr,area_name,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018
0,US,United States,"161,389,026","155,102,319","6,286,707",3.9
1,AL,Alabama,"2,216,627","2,130,845","85,782",3.9
69,AK,Alaska,"352,662","329,901","22,761",6.5
102,AZ,Arizona,"3,422,583","3,260,518","162,065",4.7
118,AR,Arkansas,"1,353,305","1,303,995","49,310",3.6


In [8]:
load_poverty = ("Resources/PovertyReport.csv")
poverty_df = pd.read_csv(load_poverty)

In [9]:
poverty_df = poverty_df.drop(columns={"Textbox98", "Textbox99", "total_est_pct_Bmin2",
                                     "total_est_pct_Bmax2", "under18_pct_Bmin2", "under18_pct_Bmax2"})
poverty_df = poverty_df.rename(columns={"state_National": "area_name",
                                       "total_est_pct2": "Poverty (2018)",
                                       "under18_pct2": "Under 18 in Poverty (2018)"})
poverty_df.head()

,area_name,Poverty (2018),Under 18 in Poverty (2018)
0,Alabama,16.8,23.9
1,Alaska,11.1,14.5
2,Arizona,14.1,20.4
3,Arkansas,16.8,23.8
4,California,12.8,17.4


In [10]:
#add and read the hate crime file
hate_crime = ("Resources/hate_crimes538.csv")
hate_crime_df = pd.read_csv(hate_crime)
hate_crime_df.head()

,state,median_household_income,share_unemployed_seasonal,share_population_in_metro_areas,share_population_with_high_school_degree,share_non_citizen,share_white_poverty,gini_index,share_non_white,share_voters_voted_trump,hate_crimes_per_100k_splc,avg_hatecrimes_per_100k_fbi
0,Alabama,42278,0.060,0.64,0.821,0.02,0.12,0.472,0.35,0.63,0.125839,1.806410
1,Alaska,67629,0.064,0.63,0.914,0.04,0.06,0.422,0.42,0.53,0.143740,1.656700
2,Arizona,49254,0.063,0.90,0.842,0.10,0.09,0.455,0.49,0.50,0.225320,3.413928
3,Arkansas,44922,0.052,0.69,0.824,0.04,0.12,0.458,0.26,0.60,0.069061,0.869209
4,California,60487,0.059,0.97,0.806,0.13,0.09,0.471,0.61,0.33,0.255805,2.397986


In [11]:
hate_crime_colm = hate_crime_df[["state", "median_household_income", "hate_crimes_per_100k_splc"]]
hate_crime_colm = hate_crime_colm.rename(columns={"state" : "area_name"})
hate_crime_colm.head()

,area_name,median_household_income,hate_crimes_per_100k_splc
0,Alabama,42278,0.125839
1,Alaska,67629,0.143740
2,Arizona,49254,0.225320
3,Arkansas,44922,0.069061
4,California,60487,0.255805


In [12]:
unempl_data["Average Unempl 2014-18"] =(unempl_data["Unemployment_rate_2018"] + unempl_data["Unemployment_rate_2017"]
+ unempl_data["Unemployment_rate_2016"] + unempl_data["Unemployment_rate_2015"] + 
unempl_data["Unemployment_rate_2014"])/4
unempl_data.head()

,FIPStxt,Stabr,area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,...,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Median_Household_Income_2018,Med_HH_Income_Percent_of_State_Total_2018,Average Unempl 2014-18
0,0,US,United States,NaN,NaN,NaN,"142,601,667","136,904,680","5,696,987",4.0,...,"155,102,319","6,286,707",3.9,"163,100,055","157,115,247","5,984,808",3.7,"61,937",NaN,6.100
1,1000,AL,Alabama,NaN,NaN,NaN,"2,133,223","2,035,594","97,629",4.6,...,"2,130,845","85,782",3.9,"2,241,747","2,174,483","67,264",3.0,"49,881",100.0,6.725
2,1001,AL,"Autauga County, AL",2.0,2.0,1.0,"21,720","20,846",874,4.0,...,"25,261",935,3.6,"26,172","25,458",714,2.7,"59,338",119.0,5.850
3,1003,AL,"Baldwin County, AL",3.0,2.0,1.0,"69,533","66,971","2,562",3.7,...,"91,809","3,424",3.6,"97,328","94,675","2,653",2.7,"57,588",115.5,6.275
4,1005,AL,"Barbour County, AL",6.0,6.0,0.0,"11,373","10,748",625,5.5,...,"7,987",427,5.1,"8,537","8,213",324,3.8,"34,382",68.9,9.675


In [13]:
edu_data_final = edu_data_upd.drop([0,1], axis=0)
edu_data_final.head()

,area_name,Total Edu,Total Edu Urban,Total Edu Rural
2,Alabama,24.90%,28.00%,15.20%
3,Alaska,29.20%,31.80%,23.90%
4,Arizona,28.90%,29.60%,15.80%
5,Arkansas,22.60%,26.80%,16.00%
6,California,33.30%,33.50%,23.20%


In [14]:
unempl_data['Rural_urban_continuum_code_2013'].replace('', np.nan, inplace=True)
unempl_state_interim = unempl_data[unempl_data['Rural_urban_continuum_code_2013'].isnull()]
unempl_state_interim
unempl_data_clean = unempl_state_interim[unempl_state_interim['Metro_2013']!=0]
unempl_data_clean.head()

,FIPStxt,Stabr,area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,...,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Median_Household_Income_2018,Med_HH_Income_Percent_of_State_Total_2018,Average Unempl 2014-18
0,0,US,United States,NaN,NaN,NaN,"142,601,667","136,904,680","5,696,987",4.0,...,"155,102,319","6,286,707",3.9,"163,100,055","157,115,247","5,984,808",3.7,"61,937",NaN,6.100
1,1000,AL,Alabama,NaN,NaN,NaN,"2,133,223","2,035,594","97,629",4.6,...,"2,130,845","85,782",3.9,"2,241,747","2,174,483","67,264",3.0,"49,881",100.0,6.725
69,2000,AK,Alaska,NaN,NaN,NaN,"319,511","299,146","20,365",6.4,...,"329,901","22,761",6.5,"347,779","326,430","21,349",6.1,"74,912",100.0,8.550
102,4000,AZ,Arizona,NaN,NaN,NaN,"2,509,883","2,410,581","99,302",4.0,...,"3,260,518","162,065",4.7,"3,551,260","3,384,504","166,756",4.7,"59,079",100.0,6.950
118,5000,AR,Arkansas,NaN,NaN,NaN,"1,258,301","1,204,695","53,606",4.3,...,"1,303,995","49,310",3.6,"1,362,642","1,314,290","48,352",3.5,"47,094",100.0,5.575


In [15]:
unempl_data_mycolm.head()


,Stabr,area_name,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018
0,US,United States,"161,389,026","155,102,319","6,286,707",3.9
1,AL,Alabama,"2,216,627","2,130,845","85,782",3.9
69,AK,Alaska,"352,662","329,901","22,761",6.5
102,AZ,Arizona,"3,422,583","3,260,518","162,065",4.7
118,AR,Arkansas,"1,353,305","1,303,995","49,310",3.6


In [16]:
# Merge the Education Dataframe with Enemployment Dataframe
merged_df = pd.merge(edu_data_final,unempl_data_mycolm, on="area_name")
merged_df.head()

,area_name,Total Edu,Total Edu Urban,Total Edu Rural,Stabr,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018
0,Alabama,24.90%,28.00%,15.20%,AL,"2,216,627","2,130,845","85,782",3.9
1,Alaska,29.20%,31.80%,23.90%,AK,"352,662","329,901","22,761",6.5
2,Arizona,28.90%,29.60%,15.80%,AZ,"3,422,583","3,260,518","162,065",4.7
3,Arkansas,22.60%,26.80%,16.00%,AR,"1,353,305","1,303,995","49,310",3.6
4,California,33.30%,33.50%,23.20%,CA,"19,280,821","18,460,725","820,096",4.3


In [17]:
#add Poverty Dataframe to the merged data
merged_df1 = pd.merge(merged_df, poverty_df, on="area_name")
merged_df1.head(3)

,area_name,Total Edu,Total Edu Urban,Total Edu Rural,Stabr,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Poverty (2018),Under 18 in Poverty (2018)
0,Alabama,24.90%,28.00%,15.20%,AL,"2,216,627","2,130,845","85,782",3.9,16.8,23.9
1,Alaska,29.20%,31.80%,23.90%,AK,"352,662","329,901","22,761",6.5,11.1,14.5
2,Arizona,28.90%,29.60%,15.80%,AZ,"3,422,583","3,260,518","162,065",4.7,14.1,20.4


In [24]:
#ready for the final code
#add the Hate Crime data to the merged dataframe

merged_df_fin = pd.merge(merged_df1,hate_crime_colm, on="area_name")
merged_df_fin.head()

,area_name,Total Edu,Total Edu Urban,Total Edu Rural,Stabr,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Poverty (2018),Under 18 in Poverty (2018),median_household_income,hate_crimes_per_100k_splc
0,Alabama,24.90%,28.00%,15.20%,AL,"2,216,627","2,130,845","85,782",3.9,16.8,23.9,42278,0.125839
1,Alaska,29.20%,31.80%,23.90%,AK,"352,662","329,901","22,761",6.5,11.1,14.5,67629,0.143740
2,Arizona,28.90%,29.60%,15.80%,AZ,"3,422,583","3,260,518","162,065",4.7,14.1,20.4,49254,0.225320
3,Arkansas,22.60%,26.80%,16.00%,AR,"1,353,305","1,303,995","49,310",3.6,16.8,23.8,44922,0.069061
4,California,33.30%,33.50%,23.20%,CA,"19,280,821","18,460,725","820,096",4.3,12.8,17.4,60487,0.255805


In [59]:
fig = px.choropleth(merged_df_fin,  # Input Pandas DataFrame
                    locations="Stabr",  # DataFrame column with locations
                    color="Total Edu Urban",  # DataFrame column with color values
                    color_continuous_scale="Blues",
                    hover_name="area_name", # DataFrame column hover info
                    locationmode = 'USA-states') # Set to plot as US States
colorscale = ['#f2f0f7','#dadaeb','#bcbddc','#9e9ac8','#756bb1','#54278f']
fig.update_layout(
    title_text = 'State Rankings', # Create a Title
#     autocolorscale = True,
#     colorscale = "sequential", 
    geo_scope='usa',  # Plot only the USA instead of globe
)
fig.show()  # Output the plot to the screen

In [ ]:
# occurrence of Hate Crimes to Education (Bachelor's Degree vs. High School) by State

In [ ]:
# occurrence of Hate Crimes to Poverty Level by State

In [ ]:
# occurrence of Hate Crimes to Poverty Level by State

In [ ]:
# Clean FBI Data set - take 2014-2018 data and average hate crimes fbi